In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite


In [2]:
# Load the Aspergillus niger ATCC1015 model
model = read_sbml_model('../model/Aspergillus_ATCC1015.xml')
model

'' is not a valid SBML 'SId'.


Name,
Memory address,0x01b5e934eeb0
Number of metabolites,1818
Number of reactions,2320
Number of groups,0
Objective expression,1.0*DRAIN_Biomass - 1.0*DRAIN_Biomass_reverse_fcc30
Compartments,"Cytoplasm, Mitochondria, Extracellular, Peroxisome, Nucleus, Golgi, Vacuole"


In [3]:
# Find pathway and metabolites
for m in model.metabolites.query('Farnesyl', 'name'):
    print(m.id,'\n',m.name)

FPP 
 trans,trans-Farnesyl diphosphate


In [4]:
# Create all metabolites
Amorphadiene = Metabolite(id = 'amorpha', name = 'Amorphadiene', compartment = 'c')
Artemisinic_alcohol = Metabolite(id = 'artem_alcohol', name = 'Artemisinic alcohol', compartment = 'c')
Artemisinic_aldehyde = Metabolite(id = 'artem_aldehyde', name = 'Artemisinic aldehyde', compartment = 'c')
Artemisinic_acid_c = Metabolite(id = 'artem_acid_c', name = 'Artemisinic acid_c', compartment = 'c')
Artemisinic_acid_e = Metabolite(id = 'artem_acid_e', name = 'Artemisinic acid_e', compartment = 'e')


In [5]:
# Create all reactions ##changed name to ID in function attributes for clarification
def add_reaction (meta1, meta2, ID):
    new_reaction = Reaction(ID)
    new_reaction.add_metabolites({
        meta1: -1,
        meta2: 1
    })
    model.add_reactions([new_reaction])
    print(new_reaction.build_reaction_string(), " added.")

add_reaction(model.metabolites.FPP, Amorphadiene, 'ADS_reaction')
add_reaction(Amorphadiene, Artemisinic_alcohol, 'CYP1/AV1CYPCPR1CYB5_reaction')
add_reaction(Artemisinic_alcohol, Artemisinic_aldehyde, 'ADH1_reaction')
add_reaction(Artemisinic_aldehyde, Artemisinic_acid_c, 'ALDH1_reaction')


#create exchange reaction for secretion
add_reaction(Artemisinic_acid_c, Artemisinic_acid_e, 'secretion')
model.add_boundary(model.metabolites.get_by_id("artem_acid_e"), type="exchange")

FPP --> amorpha  added.
amorpha --> artem_alcohol  added.
artem_alcohol --> artem_aldehyde  added.
artem_aldehyde --> artem_acid_c  added.
artem_acid_c --> artem_acid_e  added.


Reaction identifier,EX_artem_acid_e
Name,Artemisinic acid_e exchange
Memory address,0x1b5fa3a3d30
Stoichiometry,artem_acid_e <=> Artemisinic acid_e <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [76]:
print('Model objective: ', model.objective)
print('Value Biomass production (Flux): ', model.optimize().objective_value)
with model:
    model.objective = model.reactions.secretion
    solution = model.optimize()   #create solution to check fluxes
   # model.solver ="glpk"
    print('\nModel objective: ', model.objective)
    print('Value of artemisinic acid secretion: ', model.optimize().objective_value)
    #print('Value of artemisinic acid secretion: ', solution.fluxes.secretion)
    print('Flux Biomass: ', solution.fluxes.DRAIN_Biomass)
    

Model objective:  Maximize
1.0*DRAIN_Biomass - 1.0*DRAIN_Biomass_reverse_fcc30
Value Biomass production (Flux):  0.9398547241492543

Model objective:  Maximize
1.0*secretion - 1.0*secretion_reverse_c5a7c
Value of artemisinic acid secretion:  2.8081343943412915
Flux Biomass:  0.0


In [80]:
#Just to fuck around. ignore
print('Model objective: ', model.objective)
print('with Biomass production as objective function: ')
solution = model.optimize()
print(solution.fluxes.r54)

with model:
    model.objective = model.reactions.r54
    print('Model objective: ', model.objective)
    print('with self production as objective function: ')
    solution = model.optimize()
    print(solution.fluxes.r54)
    
    print('\nFlux Biomass: ', solution.fluxes.DRAIN_Biomass)

Model objective:  Maximize
1.0*DRAIN_Biomass - 1.0*DRAIN_Biomass_reverse_fcc30
with Biomass production as objective function: 
-34.85190304078644
Model objective:  Maximize
1.0*r54 - 1.0*r54_reverse_01081
with self production as objective function: 
106.95652173913032

Flux Biomass:  0.0


In [81]:
#check what reaction is biomass reaction
for reaction in model.reactions.query('mass', 'id'):
    print(reaction.id, reaction, reaction.name)

DRAIN_Biomass DRAIN_Biomass: BIOMASS -->  
